In [9]:
from keras.models import Sequential
from keras.models import load_model
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils  import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from utilities import one_hot_encoding, read_train_data, read_label_data, show_train_history

SIZE = 6500
MODEL_FOLDER = "model/"
WIDTH = 200
HEIGHT = 60
PROCESSED_FOLDER = "processed/"
LABEL_CSV_FILE = 'label.csv'
allowedChars = 'ACDEFGHJKLNPQRTUVXYZ2346789';

In [10]:
# creat CNN model
print('Creating CNN model...')
tensor_in = Input((HEIGHT, WIDTH, 3))

tensor_out = tensor_in
tensor_out = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(tensor_out)
tensor_out = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(tensor_out)
tensor_out = MaxPooling2D(pool_size=(2, 2))(tensor_out)
tensor_out = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(tensor_out)
tensor_out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(tensor_out)
tensor_out = MaxPooling2D(pool_size=(2, 2))(tensor_out)
tensor_out = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(tensor_out)
tensor_out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(tensor_out)
tensor_out = BatchNormalization(axis=1)(tensor_out)
tensor_out = MaxPooling2D(pool_size=(2, 2))(tensor_out)
tensor_out = Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu')(tensor_out)
tensor_out = Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu')(tensor_out)
tensor_out = MaxPooling2D(pool_size=(2, 2))(tensor_out)
tensor_out = Conv2D(filters=512, kernel_size=(3, 3), padding='same', activation='relu')(tensor_out)
tensor_out = BatchNormalization(axis=1)(tensor_out)
tensor_out = MaxPooling2D(pool_size=(2, 2))(tensor_out)

tensor_out = Flatten()(tensor_out)
tensor_out = Dropout(0.5)(tensor_out)

tensor_out = [Dense(len(allowedChars), name='digit1', activation='softmax')(tensor_out),\
              Dense(len(allowedChars), name='digit2', activation='softmax')(tensor_out),\
              Dense(len(allowedChars), name='digit3', activation='softmax')(tensor_out),\
              Dense(len(allowedChars), name='digit4', activation='softmax')(tensor_out),\
              Dense(len(allowedChars), name='digit5', activation='softmax')(tensor_out)]


Creating CNN model...


In [11]:
model = Model(inputs=tensor_in, outputs=tensor_out)
model.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 60, 200, 3)   0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 60, 200, 32)  896         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 58, 198, 32)  9248        conv2d_10[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 29, 99, 32)   0           conv2d_11[0][0]                  
____________________________________________________________________________________________

In [12]:
print("Reading training data...")

train_data = read_train_data(PROCESSED_FOLDER, SIZE)
train_label = read_label_data(LABEL_CSV_FILE, allowedChars, 5)

print("Reading completed")

Reading training data...
Reading completed


In [13]:
filepath = MODEL_FOLDER + "{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_digit4_acc', verbose=1, save_best_only=False, mode='max')
#earlystop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, mode='auto')
tensorBoard = TensorBoard(log_dir = 'logs', histogram_freq = 1)
callbacks_list = [tensorBoard, checkpoint]

In [14]:
print('model loading...')
model = load_model("twse_cnn_model.hdf5")
print('loading completed')

model loading...
loading completed


In [15]:
history = model.fit(train_data, train_label, validation_split=0.2, batch_size=50, epochs=30, verbose=1, callbacks=callbacks_list)

Train on 5200 samples, validate on 1300 samples
Epoch 1/30
5200/5200 [==============================] - 223s 43ms/step - loss: 0.1693 - digit1_loss: 0.0275 - digit2_loss: 0.0280 - digit3_loss: 0.0259 - digit4_loss: 0.0285 - digit5_loss: 0.0593 - digit1_acc: 0.9913 - digit2_acc: 0.9933 - digit3_acc: 0.9927 - digit4_acc: 0.9919 - digit5_acc: 0.9813 - val_loss: 0.6187 - val_digit1_loss: 0.0631 - val_digit2_loss: 0.0838 - val_digit3_loss: 0.1333 - val_digit4_loss: 0.1398 - val_digit5_loss: 0.1987 - val_digit1_acc: 0.9862 - val_digit2_acc: 0.9785 - val_digit3_acc: 0.9723 - val_digit4_acc: 0.9685 - val_digit5_acc: 0.9577


InvalidArgumentError: You must feed a value for placeholder tensor 'input_1_1' with dtype float and shape [?,60,200,3]
	 [[node input_1_1 (defined at /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541) ]]

Original stack trace for 'input_1_1':
  File "/usr/local/Cellar/python/3.7.4_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.7.4_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.7/site-pac

In [ ]:
show_train_history(history, 'digit1_acc', 'val_digit1_acc')